In [ ]:
pip show azure-ai-ml

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

In [ ]:
%%writefile src/mlflow-env.yml

name: mlflow-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - pip
  - pip:
    - numpy
    - pandas
    - scikit-learn
    - matplotlib
    - mlflow
    - azureml-mlflow

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_mlflow = Environment(
    image='mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.2-cudnn8-ubuntu18.04',
    conda_file='src/mlflow-env.yml',
    name='docker-plus-mlflow',
    description='Environment with Docker and MLflow')

ml_client.environments.create_or_update(env_docker_mlflow)

In [ ]:
from azure.ai.ml import command

job=command(
    code="./src",
    command="python train-classification-model-mlflow.py --training_data diabetes.csv --reg_rate 1 --max_iter 1000",
    environment="docker-plus-mlflow:1",
    compute="aml-cluster",
    display_name="diabetes-train-script_mlflow",
    experiment_name="diabetes-training_mlflow"

)
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print(f"Job submitted to {aml_url}")

In [ ]:
job=command(
    code="./src",
    command="python train-classification-model-mlflow-auto.py --training_data diabetes.csv --reg_rate 1 --max_iter 1000",
    environment="docker-plus-mlflow:1",
    compute="aml-cluster",
    display_name="diabetes-train-script_mlflow_auto",
    experiment_name="diabetes-training_mlflow_auto"

)
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print(f"Job submitted to {aml_url}")